In [1]:
import nest_asyncio

import autogen
from autogen.agents.experimental import WebSurferAgent

nest_asyncio.apply()

In [2]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"tags": ["gpt-4o-mini"]},
)
llm_config = {"config_list": config_list}

In [3]:
websurfer = WebSurferAgent(
    llm_config=llm_config,
    name="WebSurferAgent",
    web_tool="crawl4ai",
)

# websurfer.run(
#     message="Give ma summary of the following page: https://docs.ag2.ai/docs/home/home",
#     tools=websurfer.tools,
#     max_turns=2,
#     user_input=False,
# )
for tool in websurfer.tools:
    websurfer.register_for_execution()(tool)

websurfer.function_map

{'crawl4ai': <function autogen.tools.experimental.crawl4ai.crawl4ai.Crawl4AITool.__init__.<locals>.crawl4ai_with_llm(url: typing.Annotated[str, <autogen.tools.dependency_injection.Field object at 0x10f8b5f50>], instruction: typing.Annotated[str, <autogen.tools.dependency_injection.Field object at 0x108e1b0d0>]) -> Any>}

In [4]:
# InitialAnalysisAgent
# WebsurferAgent
# SummarizerAgent

In [5]:
import json

from autogen import (
    AFTER_WORK,
    AfterWorkOption,
    ConversableAgent,
    OnCondition,
    UserProxyAgent,
    initiate_swarm_chat,
    register_hand_off,
)

# Swarm Agents, similar to ConversableAgent, but with functions and hand offs (specified later)
initial_analyser = ConversableAgent(
    name="InitialAnalysisAgent",
    system_message="""You are an expert at breaking down complex questions into smaller, focused subquestions.
Your task is to take any question provided and divide it into clear, actionable subquestions that can be individually answered.
Ensure the subquestions are logical, non-redundant, and cover all key aspects of the original question.
Avoid providing answers or interpretations—focus solely on decomposition.

Do NOT forward the initial task without breaking it down into subquestions!!!
""",
    llm_config=llm_config,
    # functions=[verify_customer_identity],
)

websurfer = WebSurferAgent(
    llm_config=llm_config,
    name="WebSurferAgent",
    web_tool="crawl4ai",
)

summarizer = ConversableAgent(
    name="SummarizerAgent",
    system_message="""You are a summarizer agent responsible for gathering and synthesizing answers to subquestions to provide a complete and concise response to the original question.
If the gathered information is insufficient to answer the original question, request further analysis from the initial analysis agent by specifying what is missing.
Otherwise, combine the information into a clear and comprehensive summary that addresses the original question.""",
    llm_config=llm_config,
    # functions=[process_refund_payment],
)

# Conditional and After work hand offs

register_hand_off(
    agent=initial_analyser,
    hand_to=[
        OnCondition(
            websurfer,
            "After you have broken down the question, do the web research one task at a time. Do NOT start the web research BEFORE breaking down the question.",
        ),
        AFTER_WORK(summarizer),
    ],
)


register_hand_off(
    agent=websurfer,
    hand_to=[
        AFTER_WORK(initial_analyser),
    ],
)

register_hand_off(
    agent=summarizer,
    hand_to=[
        OnCondition(
            initial_analyser,
            "If there are any subquestions that need further clarification, hand off to the Initial Analysis Agent",
        ),
        AFTER_WORK(AfterWorkOption.REVERT_TO_USER),
    ],
)


for tool in websurfer.tools:
    websurfer.register_for_execution()(tool)

# Our human, you, allowing swarm agents to revert back for more information
user = UserProxyAgent(name="User", code_execution_config=False)

# Initiate the swarm
# Returns the ChatResult, final context, and last speaker

chat_result, context_variables, last_speaker = initiate_swarm_chat(
    initial_agent=initial_analyser,  # Starting agent
    agents=[initial_analyser, websurfer, summarizer],  # Swarm agents
    user_agent=user,  # Human user
    messages="Which are the best cryptocurrencies to invest in? Do the web research first and make som conclusions",  # Initial message
    # context_variables=context_variables, # Context
    after_work=AFTER_WORK(AfterWorkOption.TERMINATE),  # Swarm-level after work hand off
)


print(f"Context Variables:\n{json.dumps(context_variables, indent=2)}")

/var/folders/79/kmq3y0pj0rbgrrnf4xlq8q0c0000gn/T/ipykernel_54375/3259236999.py:43: DeprecationWarning: AFTER_WORK is deprecated and will be removed in a future version (TBD). Use AfterWork instead.
  AFTER_WORK(summarizer),
/var/folders/79/kmq3y0pj0rbgrrnf4xlq8q0c0000gn/T/ipykernel_54375/3259236999.py:51: DeprecationWarning: AFTER_WORK is deprecated and will be removed in a future version (TBD). Use AfterWork instead.
  AFTER_WORK(initial_analyser),
/var/folders/79/kmq3y0pj0rbgrrnf4xlq8q0c0000gn/T/ipykernel_54375/3259236999.py:59: DeprecationWarning: AFTER_WORK is deprecated and will be removed in a future version (TBD). Use AfterWork instead.
  AFTER_WORK(AfterWorkOption.REVERT_TO_USER),
/var/folders/79/kmq3y0pj0rbgrrnf4xlq8q0c0000gn/T/ipykernel_54375/3259236999.py:79: DeprecationWarning: AFTER_WORK is deprecated and will be removed in a future version (TBD). Use AfterWork instead.
  after_work=AFTER_WORK(AfterWorkOption.TERMINATE) # Swarm-level after work hand off


User (to chat_manager):

Which are the best cryptocurrencies to invest in? Do the web research first and make som conclusions

--------------------------------------------------------------------------------

Next speaker: InitialAnalysisAgent


>>>>>>>> USING AUTO REPLY...
InitialAnalysisAgent (to chat_manager):

***** Suggested tool call (call_Pwa2b4ZWpBTGjZW8ArGoC9Ag): transfer_InitialAnalysisAgent_to_WebSurferAgent *****
Arguments: 
{}
****************************************************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Swarm_Tool_Executor


>>>>>>>> EXECUTING FUNCTION transfer_InitialAnalysisAgent_to_WebSurferAgent...
Call ID: call_Pwa2b4ZWpBTGjZW8ArGoC9Ag
Input arguments: {}
_Swarm_Tool_Executor (to chat_manager):

***** Response from calling tool (call_Pwa2b4ZWpBTGjZW8ArGoC9Ag) *****
Swarm agent --> WebSurferAgent
******************************************